In [40]:
import numpy as np
import random
import torch 
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import random

In [90]:
level_lower=np.array([182.37, 170.44, 170.33, 173.08, 73.67])
level_upper=np.array([183.5, 177.6,177.6,175,75.4])
level_lower=torch.from_numpy(level_lower).to(device)
level_upper=torch.from_numpy(level_upper).to(device)
# S M H E O
S_2018=[183.65,183.58,183.54,183.47,183.5,183.56,183.65,183.65,183.7,183.77,183.76,183.67]
M_2018=[176.73,176.74,176.76,176.79,176.92,176.98,176.98,176.94,176.94,176.89,176.86,176.83]
E_2018=[174.38,174.44,174.68,174.77,174.88,174.89,174.82,174.72,174.66,174.57,174.58,174.6]
O_2018=[74.83,74.9,74.93,75.01,75.26,75.22,75.06,74.91,74.74,74.62,74.62,74.71]
A_2018=[S_2018,M_2018,E_2018,O_2018]
def my_obj_func(model):
    def get_2018(id_lake, id_month):
        return A_2018[int(id_lake)][int(id_month)]
    xs=[float(i) for i in range(12)]
    ls=[float(i) for i in range(4)]
    ys={}
    res=torch.zeros(1).to(device).type(torch.float32)
    for x in xs:
        for l in ls:
            input_x=np.array([x,l])
            input_x=torch.from_numpy(input_x).to(device).type(torch.float32)
            ys[f"{x},{l}"]=model(input_x)*200.0
    for idx, y_key in enumerate(ys.keys()):
        lake_idx=int(float((y_key.split(",")[1])))
        if ys[y_key]>level_upper[lake_idx]:
            res+=1000*torch.abs(ys[y_key]-level_upper[lake_idx])
        elif ys[y_key]<level_lower[lake_idx]:
            res+=1000*torch.abs(ys[y_key]-level_lower[lake_idx])
    # 游客, 航运, 生态
    ws=[1,1,1]
    tor=torch.zeros(1).to(device).type(torch.float32)
    shp=torch.zeros(1).to(device).type(torch.float32)
    env=torch.zeros(1).to(device).type(torch.float32)
    for x in range(4,8):
        for l in ls:
            beta=random.gauss(177.9)
            tor+=ws[0]*torch.abs(ys[f"{float(x)},{l}"]-beta)
    res+=torch.mean(tor)
    for x in range(12):
        for l in ls:
            beta=random.gauss(187.6)
            shp+=ws[1]*torch.abs(ys[f"{float(x)},{l}"]-beta)
    res+=torch.mean(shp)
    for x in range(12):
        for l in ls:
            env+=ws[2]*torch.abs(ys[f"{float(x)},{l}"]-get_2018(l, x))
    res+=torch.mean(env)
    return res

In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the Fully Connected Neural Network
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2=nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x=torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the custom loss function (for example, MSE and L2 regularization)
def custom_loss(output, target, model, lambda_reg=0.01):
    # MSE loss
    mse_loss = my_obj_func(model)
    
    # L2 Regularization (on weights)
    l2_reg = sum(torch.sum(param ** 2) for param in model.parameters())
    
    return mse_loss + lambda_reg * l2_reg

# Particle class for MOPSO
class Particle:
    def __init__(self, model, input_size, hidden_size, output_size, lower_bound, upper_bound, device):
        self.device = device
        self.position = []
        self.velocity = []
        self.best_position = []
        self.best_objectives = []
        
        # Initialize weights (randomly within bounds)
        for param in model.parameters():
            shape = param.shape
            param_size = param.numel()
            
            # Initialize position and velocity
            self.position.append(torch.rand_like(param, device=self.device) * (upper_bound - lower_bound) + lower_bound)
            self.velocity.append(torch.rand_like(param, device=self.device) * 0.1)
            self.best_position.append(self.position[-1].clone())
            self.best_objectives.append(np.inf)  # Start with a high objective value
        
        # Flatten lists of tensors into single tensor
        self.position = torch.cat([p.view(-1) for p in self.position])
        self.velocity = torch.cat([v.view(-1) for v in self.velocity])
        self.best_position = torch.cat([bp.view(-1) for bp in self.best_position])

    def update_velocity(self, global_best_position, inertia, cognitive_weight, social_weight):
        r1, r2 = torch.rand(2, device=self.device)
        self.velocity = (inertia * self.velocity +
                         cognitive_weight * r1 * (self.best_position - self.position) +
                         social_weight * r2 * (global_best_position - self.position))

    def update_position(self, lower_bound, upper_bound):
        self.position += self.velocity
        self.position = torch.clamp(self.position, min=lower_bound, max=upper_bound)

    def update_best(self, loss_value):
        if loss_value < self.best_objectives[0]:  # Assuming single objective
            self.best_position = self.position.clone()
            self.best_objectives[0] = loss_value

# MOPSO Algorithm
class MOPSO:
    def __init__(self, model, input_size, hidden_size, output_size, population_size, lower_bound, upper_bound, inertia, cognitive_weight, social_weight, iterations, train_loader, lambda_reg=0.01, device='cuda'):
        self.model = model.to(device)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.population_size = population_size
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.inertia = inertia
        self.cognitive_weight = cognitive_weight
        self.social_weight = social_weight
        self.iterations = iterations
        self.train_loader = train_loader
        self.lambda_reg = lambda_reg
        self.device = device

        # Initialize the particles
        self.particles = [Particle(model, input_size, hidden_size, output_size, lower_bound, upper_bound, device) for _ in range(population_size)]
        self.global_best_position = self.particles[0].best_position

    def run(self):
        for iteration in range(self.iterations):
            for particle in self.particles:
                # Set model parameters to the particle's position (flattened)
                idx = 0
                with torch.no_grad():
                    for param in self.model.parameters():
                        param_size = param.numel()
                        param.data = particle.position[idx:idx+param_size].view(param.shape)
                        idx += param_size

                # Calculate loss using the model
                total_loss = 0
                for inputs, targets in self.train_loader:
                    inputs, targets = inputs.to(self.device), targets.to(self.device)
                    outputs = self.model(inputs)
                    loss = custom_loss(outputs, targets, self.model, lambda_reg=self.lambda_reg)
                    total_loss += loss.item()

                # Update particle's best
                particle.update_best(total_loss)

            # Update global best position
            self.global_best_position = min(self.particles, key=lambda p: p.best_objectives[0]).best_position

            # Update particles' velocities and positions
            for particle in self.particles:
                particle.update_velocity(self.global_best_position, self.inertia, self.cognitive_weight, self.social_weight)
                particle.update_position(self.lower_bound, self.upper_bound)

            # Print the progress
            print(f"Iteration {iteration+1}/{self.iterations} - Global Best Loss: {min([p.best_objectives[0] for p in self.particles])}")

# Example usage
if __name__ == "__main__":
    # Set up training data (dummy dataset for illustration)
    input_size = 2
    hidden_size = 5
    output_size = 1
    train_data = torch.randn(100, input_size)
    train_data[:,0]*=11.0
    train_data[:,1]*=3.0
    train_targets = torch.randn(100, output_size)
    train_loader = torch.utils.data.DataLoader(list(zip(train_data, train_targets)), batch_size=10)

    # Define the model
    model = FCNN(input_size, hidden_size, output_size)

    # MOPSO parameters
    population_size = 30
    lower_bound = -0.5
    upper_bound = 0.5
    inertia = 0.7
    cognitive_weight = 1.5
    social_weight = 1.5
    iterations = 150

    # Device configuration (set to "cuda" if GPU is available, otherwise "cpu")
    

    # Run MOPSO
    mopso = MOPSO(model, input_size, hidden_size, output_size, population_size, lower_bound, upper_bound, inertia, cognitive_weight, social_weight, iterations, train_loader, device=device)
    mopso.run()


Iteration 1/50 - Global Best Loss: 22270240.75
Iteration 2/50 - Global Best Loss: 19859985.0
Iteration 3/50 - Global Best Loss: 12772341.25
Iteration 4/50 - Global Best Loss: 6908765.25
Iteration 5/50 - Global Best Loss: 6640177.9375
Iteration 6/50 - Global Best Loss: 6084593.5
Iteration 7/50 - Global Best Loss: 6084593.5
Iteration 8/50 - Global Best Loss: 6084593.5
Iteration 9/50 - Global Best Loss: 5753969.0
Iteration 10/50 - Global Best Loss: 5256085.125
Iteration 11/50 - Global Best Loss: 3280226.90625
Iteration 12/50 - Global Best Loss: 2867230.875
Iteration 13/50 - Global Best Loss: 2867230.875
Iteration 14/50 - Global Best Loss: 2698800.90625
Iteration 15/50 - Global Best Loss: 2234988.984375
Iteration 16/50 - Global Best Loss: 2234988.984375
Iteration 17/50 - Global Best Loss: 2234988.984375
Iteration 18/50 - Global Best Loss: 2203533.0
Iteration 19/50 - Global Best Loss: 2165320.578125
Iteration 20/50 - Global Best Loss: 1594104.703125
Iteration 21/50 - Global Best Loss: 12945

In [97]:
test_S=[]
test_M=[]
test_E=[]
test_O=[]
def construct_test(var:list, lake_id:int):
    for mm in range(12):
        var.append([mm,lake_id])
construct_test(test_S,0)
construct_test(test_M,1)
construct_test(test_E,2)
construct_test(test_O, 3)
# S M E O
test_S=torch.from_numpy(np.array(test_S)).to(device=device).type(torch.float32)
test_M=torch.from_numpy(np.array(test_M)).to(device=device).type(torch.float32)
test_E=torch.from_numpy(np.array(test_E)).to(device=device).type(torch.float32)
test_O=torch.from_numpy(np.array(test_O)).to(device=device).type(torch.float32)
res_S=[]
res_M=[]
res_E=[]
res_O=[]
for data in test_S:
    res_S.append((model(data)*200).cpu().item())
for data in test_M:
    res_M.append((model(data)*200).cpu().item())
for data in test_E:
    res_E.append((model(data)*200).cpu().item())
for data in test_O:
    res_O.append((model(data)*200).cpu().item())

In [101]:
print(len(res_S))

12


In [108]:
import plotly.graph_objects as go
xs=[i for i in range(12)]
def debug_plot(xx,yy):
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=xx,y=yy, mode="lines+markers"))
    fig.show()
debug_plot(xx=xs,yy=res_O)